# 3. Table Operations

_This is a Jupyter Notebook file, make sure to run all the cells in the presented order to avoid any errors_

This tutorial details how to retrieve and modify data from ETABS model DataBase Tables using `PyCSI`. This example uses an ETABS model; however, connecting to a SAP2000 or SAFE model is done by changing only the ETABSmodel class for the desired software.

## Opening example model

See the [Basic Functions](2_Basic_functions.ipynb) tutorial for details of the following code

In [1]:
# Python packages imports
import os
import sys

try:
    # PyCSI import
    import pyCSI
    
except ImportError:    
    ###################################################################
    # The following lines are only necessary for these tutorials
    from pathlib import Path
    pyCSI_dir = str(Path(os.getcwd()).parents[2])
    os.chdir(pyCSI_dir)
    import pyCSI
    ###################################################################

# example_model location 
# DO NOT MODIFY THE FOLLOWING LINES
VERSION = 21
EXAMPLE_LOCATION = os.sep.join([pyCSI_dir, 'docs', 'tutorials', 'resources', 'example_model'])
EXAMPLE_NAME = 'pyCSI_example - v' + str(VERSION) + '.EDB'
EXAMPLE_FILE = os.sep.join([EXAMPLE_LOCATION, EXAMPLE_NAME])
open_file = False  # Flag to be used in the next step

model = pyCSI.ETABSModel(version=VERSION)
try:
    # Calling get_model() with no arguments connects to an active model
    model.get_model()
    file_name = model.get_file_name()  # Get the name of the connected model

    # Check that the connected model is the pyCSI_example - v21.EDB file
    # If not, set the flag to open the model in the next step
    if not file_name == EXAMPLE_NAME:
        print('Not connected to Example Model')
        open_file = True
    
except pyCSI.APIConnectionError:
    # If no active model is found, an Attribute error is raised
    # Set the flag to open the model in the next step
    print('No active model found')
    open_file = True

if open_file:
    print('Opening example model')
    model.get_model(active_model=False, file_location=EXAMPLE_FILE)


####################################################################
DEGENKOLB ENGINEERS
You are using PyCSI v0.1
Questions or comments contact Luis Pancardo
####################################################################

No active model found
Opening example model
Successfully connected to pyCSI_example - v21.EDB


## Table property

`PyCSI` provides the `.tables` property to access all the DataBase Table methods. The following sections detail the available methods.  

## Get available tables in the model

ETABS has a large number of tables, each of which has a unique Table Key Identifier.

The `.tables.get_available_tables()` method allows one to get a list that contains the Table Keys of all the available tables in the model. You can use these keys to access the table data.

**Note**: This method list the tables that contain data at the moment of the request. This means that if no data exists for a specific table, for example the "Beam Forces" when the model is not run, then this Table Key will not appear in the returned list.

In [2]:
# Unlock the model
model.lock = False

# Available tables when model is unlocked
table_names = model.tables.get_available_tables()
print(f'Found {len(table_names)} available tables when model is unlocked', end='\n\n')

# Start analysis
model.analysis.run_analysis()

# Available tables when model is locked
table_names = model.tables.get_available_tables()
print(f'Found {len(table_names)} available tables when model is run', end='\n\n')

# Some of the available tables
n = 20
print(f'First {n} available tables')
print(*table_names[:n], sep='\n', end='\n\n')

Found 100 available tables when model is unlocked

Running analysis...
Analysis finished!
Found 143 available tables when model is run

First 20 available tables
Analysis Messages
Analysis Options - Active Degrees of Freedom
Analysis Options - Analysis Model for Nonlinear Hinges
Analysis Options - Automatic Mesh Settings for Floors
Analysis Options - Automatic Rectangular Mesh Options for Walls
Analysis Options - Cracking Analysis Options
Analysis Options - Design and Response Recovery Options
Analysis Options - SAPFire Options
Area Assignments - Additional Mass
Area Assignments - Auto Edge Constraints
Area Assignments - Diaphragms
Area Assignments - Floor Auto Mesh Options
Area Assignments - Insertion Point
Area Assignments - Local Axes
Area Assignments - Section Properties
Area Assignments - Summary
Area Load Assignments - Uniform
Area Section Property Definitions - Summary
Assembled Joint Masses
Base Reactions



## Accessing table data

You can access the data of a DataBase table only by providing its Table Key.

Use the `.tables.get_table_dataframe(table_key)` to retrieve the data of a table. The data is provided in a Pandas DataFrame format.

In [3]:
TABLE_KEY = 'Story Forces'
story_forces = model.tables.get_table_dataframe(TABLE_KEY)
print(TABLE_KEY)
display(story_forces)


Story Forces


,Story,OutputCase,CaseType,Location,P,VX,VY,T,MX,MY
0,Roof,Dead,LinStatic,Top,814850.63,0,0,0,4.400193e+08,-664918116
1,Roof,Dead,LinStatic,Bottom,937486.74,0,0,0,5.062428e+08,-764989183
2,Roof,Live,LinStatic,Top,997360.00,0,0,0,5.385744e+08,-813845760
3,Roof,Live,LinStatic,Bottom,997360.00,0,0,0,5.385744e+08,-813845760
4,Roof,Superdead,LinStatic,Top,623350.00,0,0,0,3.366090e+08,-508653600
...,...,...,...,...,...,...,...,...,...,...
123,Level 1,[LRFD 2] 1.2D+1.6L+0.5Lr,Combination,Bottom,15103590.88,0,0,0,8.155939e+09,-12324530154
124,Level 1,[LRFD 2] 1.2D+1.6LR+0.5L,Combination,Top,10563715.21,0,0,0,5.704406e+09,-8619991610
125,Level 1,[LRFD 2] 1.2D+1.6LR+0.5L,Combination,Bottom,10715206.88,0,0,0,5.786212e+09,-8743608810
126,Level 1,D+L,Combination,Top,10110150.86,0,0,0,5.459481e+09,-8249883105


### Getting all fields in the table

By default, ETABS omits columns with no data in the table. For example, The 'Step Type' column on the Story Forces table is not provided if there is no Response Spectrum Load Case.

To get all the headers in the table, regardless of its content, provide the argument `include_all_headers = True` to the `.tables.get_table_dataframe()` method.

In [4]:
TABLE_KEY = 'Story Forces'
story_forces = model.tables.get_table_dataframe(TABLE_KEY, include_all_headers=True)
print(TABLE_KEY)
display(story_forces)


Story Forces


,Story,OutputCase,CaseType,StepType,StepNumber,StepLabel,Location,P,VX,VY,T,MX,MY
0,Roof,Dead,LinStatic,NaN,NaN,NaN,Top,814850.63,0,0,0,4.400193e+08,-664918116
1,Roof,Dead,LinStatic,NaN,NaN,NaN,Bottom,937486.74,0,0,0,5.062428e+08,-764989183
2,Roof,Live,LinStatic,NaN,NaN,NaN,Top,997360.00,0,0,0,5.385744e+08,-813845760
3,Roof,Live,LinStatic,NaN,NaN,NaN,Bottom,997360.00,0,0,0,5.385744e+08,-813845760
4,Roof,Superdead,LinStatic,NaN,NaN,NaN,Top,623350.00,0,0,0,3.366090e+08,-508653600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Level 1,[LRFD 2] 1.2D+1.6L+0.5Lr,Combination,NaN,NaN,NaN,Bottom,15103590.88,0,0,0,8.155939e+09,-12324530154
124,Level 1,[LRFD 2] 1.2D+1.6LR+0.5L,Combination,NaN,NaN,NaN,Top,10563715.21,0,0,0,5.704406e+09,-8619991610
125,Level 1,[LRFD 2] 1.2D+1.6LR+0.5L,Combination,NaN,NaN,NaN,Bottom,10715206.88,0,0,0,5.786212e+09,-8743608810
126,Level 1,D+L,Combination,NaN,NaN,NaN,Top,10110150.86,0,0,0,5.459481e+09,-8249883105


### Get data from a specific group

When using ETABS, you can get table results only for the objects you have selected at the moment. You can achieve a similar behavior by providing the name of an ETABS group in the `group` argument. Doing this, the returned table will contain only the data for the objects in the specified group.

The example model has a defined group, `Roof cols GL 4`, which contains only the columns at grid line 4 located at the Roof level. The following code gets the frame properties for these specific columns.


In [5]:
TABLE_KEY = 'Frame Assignments - Summary'
frame_assignments = model.tables.get_table_dataframe(TABLE_KEY, group='Roof cols GL 4')
print(TABLE_KEY)
display(frame_assignments)


Frame Assignments - Summary


,Story,Label,UniqueName,Type,Length,AnalysisSect,DesignSect,AxisAngle,MaxStaSpcg,MinNumSta,Releases
0,Roof,C4,4,Column,204,W36X361,W36X361,NaN,NaN,3,NaN
1,Roof,C8,8,Column,204,W36X361,W36X361,NaN,NaN,3,NaN
2,Roof,C12,12,Column,204,W36X361,W36X361,NaN,NaN,3,NaN
3,Roof,C16,16,Column,204,W36X361,W36X361,NaN,NaN,3,NaN
4,Roof,C20,20,Column,204,W36X361,W36X361,NaN,NaN,3,NaN


### Get data from specific load patterns, load cases and load combos

ETABS gives the option to get table data for specific Load Patterns (LPs), Load Cases (LCs) and Load Combinations (LCombos) through its 'Show Tables' format.

<img src="..\..\images\tutorials\show_tables.png" alt="Show Tables Format" width="600"/>

In `PyCSI`, you can achieve this by modifying the `.tables.load_patterns`, `.tables.load_cases` or `.tables.load_combos` properties.

These properties accept the following values:
* A list containing the names of the LPs, LCs or LCombos to be included in the tables
* 'all' to select all defined LPs, LCs or LCombos
* None to not include any LP, LC or LCombo

In [6]:
model.tables.load_patterns = 'all'  # Include all load patterns
print(f'{model.tables.load_patterns=}', end='\n\n')

model.tables.load_cases = None  # Do not include any load case in the table
print(f'{model.tables.load_cases=}', end='\n\n')

model.tables.load_combos = ['[ASD 2] D+L', '[LRFD 1] 1.4D']  # Include only specified load combinations
print(f'{model.tables.load_combos=}', end='\n\n')



model.tables.load_patterns=['Dead', 'Live', 'Superdead', 'SX', 'Live Roof', 'SXN', 'SXP', 'SY', 'SYN', 'SYP']

model.tables.load_cases=[]

model.tables.load_combos=['[ASD 2] D+L', '[LRFD 1] 1.4D']



All subsequent calls to .tables.get_table_dataframe() after assigning a value to the .load_patterns, .load_cases or .load_combinations properties will contain the specified LPs, LCs and LCombos.

In [7]:
TABLE = 'Story Forces'
story_forces = model.tables.get_table_dataframe(TABLE)
print(TABLE_KEY)
display(story_forces)


Frame Assignments - Summary


,Story,OutputCase,CaseType,Location,P,VX,VY,T,MX,MY
0,Roof,[ASD 2] D+L,Combination,Top,2435560.63,0,0,0,1315202742,-1987417476
1,Roof,[ASD 2] D+L,Combination,Bottom,2558196.74,0,0,0,1381426242,-2087488543
2,Roof,[LRFD 1] 1.4D,Combination,Top,2013480.89,0,0,0,1087279678,-1643000403
3,Roof,[LRFD 1] 1.4D,Combination,Bottom,2185171.44,0,0,0,1179992578,-1783099896
4,Level 3,[ASD 2] D+L,Combination,Top,4993757.38,0,0,0,2696628983,-4074906019
5,Level 3,[ASD 2] D+L,Combination,Bottom,5116393.49,0,0,0,2762852483,-4174977086
6,Level 3,[LRFD 1] 1.4D,Combination,Top,4198652.33,0,0,0,2267272256,-3426100299
7,Level 3,[LRFD 1] 1.4D,Combination,Bottom,4370342.88,0,0,0,2359985156,-3566199792
8,Level 2,[ASD 2] D+L,Combination,Top,7551954.12,0,0,0,4078055225,-6162394562
9,Level 2,[ASD 2] D+L,Combination,Bottom,7674590.23,0,0,0,4144278725,-6262465628


## Edit DataBase Tables

**Coming soon...**

## Next -> [Group Objects](4_Group_Methods.ipynb)

**REMEMBER to shut down the Kernel before leaving**

Click `Kernel -> Shut Down Kernel`

### Contact

For questions or comments please reach out to:

* Luis Pancardo: [lpancardo@degenkolb.com](lpancardo@degenkolb.com)<br/>
* Daniel Gaspar:  [dgaspar@degenkolb.com](dgaspar@degenkolb.com)<br/>
